# GP Initial Implementation: GPyTorch

In [4]:
# Import needed libraries and modules
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, accuracy_score
import torch
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import gpytorch
from torch.optim import Adam

# Fetch dataset from UCI Repository
from ucimlrepo import fetch_ucirepo
heart_disease = fetch_ucirepo(id=45)
df = heart_disease.data.original

In [5]:

# ---------------------------------------------------------------------------- #
#                                PRE-PROCESSING                                #
# ---------------------------------------------------------------------------- #

# --------------------------------- SETTINGS --------------------------------- #
Normalize = True
PC_Features = True
Test_Size = 0.2
Random_Seed = 82024
Torch = True
Num_iterations = 100
Cross_Validation = True

# Drop missing values
df = df.dropna()
df = df.reset_index(drop=True)

# Binarize target
df.loc[df['num'] != 0, 'num'] = 1

# Define features and target vectors
X = df.iloc[:,:-1]
y = df['num']

# Normalize if requested
if (Normalize) or (PC_Features):
    int_features, cat_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak'],\
    ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('int', StandardScaler(), int_features),
        ('cat', OneHotEncoder(), cat_features)
    ])
    X = preprocessor.fit_transform(X)
else:
    X = X.values

# Apply PCA if requested
if PC_Features:
    pca = PCA(n_components=12)
    X = pca.fit_transform(X)
    
# Convert to torch tensor if requested
if Torch:
    X = torch.tensor(X)
    y = torch.tensor(y).double()

if not Cross_Validation:
    # Split train and test data
    index = list(range(y.shape[-1]))
    train_index, test_index = train_test_split(index, test_size=Test_Size, random_state=Random_Seed)

    train_X = X[train_index]
    train_y = y[train_index]

    test_X = X[test_index]
    test_y = y[test_index]

In [6]:
# ---------------------------------------------------------------------------- #
#                                     MODEL                                    #
# ---------------------------------------------------------------------------- #
# Create model class
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_X, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_X, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

if Cross_Validation:
    kf = KFold(shuffle=True, random_state=Random_Seed)
    roc_aucs = []
    accuracies = []

    for train_index, test_index in kf.split(X):
        train_X, test_X = X[train_index], X[test_index]
        train_y, test_y = y[train_index], y[test_index]
        
        # Initialize model
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        model = ExactGPModel(train_X, train_y, likelihood)
        
        optimizer = Adam([{'params': model.parameters()}], lr = 0.1)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

        # Train the model
        for _ in range(Num_iterations):
            optimizer.zero_grad()
            output = model(train_X)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()

        # Evaluate on validation set
        model.eval()
        likelihood.eval()
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            test_pred = likelihood(model(test_X))
            pred_probs = test_pred.mean.numpy()
            roc_auc = roc_auc_score(test_y.numpy(), pred_probs)
            accuracy = accuracy_score(test_y.numpy(), (pred_probs > 0.5).astype(int))
            
            roc_aucs.append(roc_auc)
            accuracies.append(accuracy)

    # Calculate mean metrics
    roc_auc = np.mean(roc_aucs)
    acc = np.mean(accuracies)
    
else:
    # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_X, train_y, likelihood)

    # Train model
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=0.1)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    loss_list = []
    for i in range(Num_iterations):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_X)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        loss_list.append(loss.tolist())
        optimizer.step()
        
    # Plot loss values
    plt.plot(loss_list)
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    # Test model
    model.eval()
    likelihood.eval()

    with torch.no_grad():
        y_pred = likelihood(model(test_X))
        pred_probs = y_pred.mean.numpy()

        # Convert probabilities to binary predictions using 0.5 threshold
        y_pred = (pred_probs >= 0.5).astype(int)

    # Evaluation
    acc = accuracy_score(test_y, y_pred)
    roc_auc = roc_auc_score(test_y.numpy(), pred_probs)

print(f"Accuracy: {acc:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

Accuracy: 0.8386
ROC AUC: 0.9136
